In [1]:
import pyNeuroChem as pyc
import hdnntools as hdt
import numpy as np
import time

from ase_interface import ANIENS
from ase_interface import ensemblemolecule

from ase.atoms import Atoms
from ase.optimize import BFGS, LBFGS

import pyaniasetools as pya

import batch_optimizers as opt

van Der Waals correction will be unavailable. Please install ased3


In [2]:
mol = hdt.readxyz2('/nh/nest/u/jsmith/Research/confs_test/methane.xyz')
N = 1000

wkdir = '/nh/nest/u/jsmith/Research/confs_test/arc9/'
cns = wkdir + 'rHCNO-4.6R_16-3.1A_a4-8.params'
sae = wkdir + 'sae_linfit.dat'
nnf = wkdir + 'train'
Nn = 5

In [3]:
Xl = np.stack([mol[0][0] for i in range(N)])
S = list(mol[1])
Xl.shape

(1000, 5, 3)

In [4]:
X = Xl+np.random.uniform(-0.15,0.15,size=Xl.shape)
X = np.asarray(X, dtype=np.float32)
hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify1.xyz',X,S)

In [5]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)

In [6]:
start_time = time.time()
for i,x in enumerate(X[0:1]):
    mol = Atoms(S,x)
    mol.set_calculator(ANIENS(aens))
    
    dyn = LBFGS(mol)
    dyn.run(fmax=0.00018)
    print('Molecule',i,'complete')
print('[ANI Total time:', time.time() - start_time, 'seconds]')

       Step     Time          Energy         fmax
LBFGS:    0 08:55:11    -1101.508595        4.6637
[[-0.0417264   0.10118541 -0.08386135]
 [ 0.70316428  0.69215059  0.64883101]
 [-0.52409232 -0.60904235  0.59153974]
 [-0.72452837  0.65220642 -0.72415775]
 [ 0.60304183 -0.69463539 -0.74945611]]
[[-0.00665264  0.08255643 -0.07908719]
 [ 0.68734081  0.68307874  0.62887856]
 [-0.52951397 -0.60662291  0.59050403]
 [-0.72463135  0.65447382 -0.72254178]
 [ 0.58931616 -0.67162139 -0.73485808]]
LBFGS:    1 08:55:11    -1101.802739        2.6005
[[-0.00665264  0.08255643 -0.07908719]
 [ 0.68734081  0.68307874  0.62887856]
 [-0.52951397 -0.60662291  0.59050403]
 [-0.72463135  0.65447382 -0.72254178]
 [ 0.58931616 -0.67162139 -0.73485808]]
[[ 0.01724898  0.06335983 -0.07910352]
 [ 0.66712237  0.669795    0.59834855]
 [-0.5346938  -0.5992402   0.58614969]
 [-0.70901418  0.64752462 -0.70697018]
 [ 0.57519565 -0.63957457 -0.71552901]]
LBFGS:    2 08:55:11    -1102.000643        1.3581
[[ 0.01724898

In [7]:
#molopt = opt.moleculeOptimizer(cns, sae, nnf, Nn, 0)

In [8]:
#start_time = time.time()
#X_it = molopt.optimizeGradientDescent(X,S,alpha=0.0004,convergence=1.0, printer=False)
#X_it = molopt.optimizeConjugateGradient(X_it,S,alpha=0.0001,convergence=0.005, maxsteps=5000, printstep=100, printer=False)
#print('[Complete Optimization Time:',time.time()-start_time,'s]')
#hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify_mol1.xyz',np.array(X_it),S)

In [9]:
#cnfopt = opt.conformerOptimizer(cns, sae, nnf, Nn, 0)

In [10]:
#start_time = time.time()
#X_it = cnfopt.optimizeConjugateGradient(X,S,alpha=0.0004,convergence=0.005, maxsteps=10000, printstep=100)
#print('[Complete Optimization Time:',time.time()-start_time,'s]')
#hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify_mol2.xyz',np.array(X_it),S)